# Linear Programming Series — Linear Optimization Using Popular Python Packages

## Linear Programming Using Python and The Simplex Method


### By: Francis Adrian Viernes ©

For the benefit of first-time readers, let us copy this portion from the original article.

“A bond portfolio manager has $100,000 to allocate to two different bonds; one corporate and one government bond. The corporate bond has a yield of 4%, a maturity of 3 years, and an A rating from a rating agency that is translated into a numerical rating of 2 for computational purposes.

In contrast, the government bond has a yield of 3%, a maturity of 4 years, and a rating of Aaa with the corresponding numerical rating of 1 (lower numerical ratings correspond to higher quality bonds). The portfolio manager would like to allocate her funds so that the average rating for the portfolio is no worse than Aa (numerical equivalent 1.5) and the average maturity of the portfolio is at most 3.6 years.

Any amount not invested in the two bonds will be kept in a cash account that is assumed to earn no interest, for simplicity and does not contribute to the average rating or maturity computations.

How should the manager allocate her funds between these two bonds to achieve her objective of maximizing the yield from this investment?”

# SCIPY IMPLEMENTATION

In [4]:
!pip install scipy

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063

[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
  Obtaining dependency information for pip from https://files.pythonhosted.org/packages/50/c2/e06851e8cc28dcad7c155f4753da8833ac06a5c704c109313b8d5a62968a/pip-23.2.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 26.5 MB/s eta 0:00:0000:0100:01
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming 

`SciPy` is a library that is built on top of `NumPy` . It makes sense to start with this package as familiarity with the NumPy library makes it easy to work with this library. In many of the functions inside the NumPy library, the expected input and output is an NumPy array. The SciPy library is considered mature and well-maintained although it may not be the best option to use for advanced or larger-scale optimization problems.

In [14]:
from scipy.optimize import linprog

c = [-0.04, -0.03]  # Negative for maximization problem

#A represents the coefficient of the constraints
A = [[1, 1], 
     [3, 4], 
     [2, 1]]

#b represents the right hand side of the constraints
b = [100000, 3600000, 150000]

#bounds ensure that the allocation for either variable do not go lower a certain number or beyond a certain number
#bounds are used for constraints like budget constraints
# bounds = [(0, 100000), (0, 100000)]

res = linprog(c, A_ub=A, b_ub=b, bounds=bounds, method='simplex')
print(f"Optimal Investment in Corporate Bond: ${res.x[0]}")
print(f"Optimal Investment in Government Bond: ${res.x[1]}")
print(f"Maximum Yield: {-res.fun/1000}%")

Optimal Investment in Corporate Bond: $50000.0
Optimal Investment in Government Bond: $50000.0
Maximum Yield: 3.5%


# CVXPY IMPLEMENTATION

`cvxpy` is a package designed for convex optimization. As a review, remember that if an objective function is convex and we are aiming to minimize it, any local minimizer we find is also a global minimizer. 

A huge advantage to cvxpy is that it allows users to write the code in a way that most resembles the mathemtical form or the standard form. 

In [15]:
!pip install cvxpy

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [20]:
import cvxpy as cp

# Define the decision variables
x_corp = cp.Variable()  # Amount invested in the corporate bond
x_gov = cp.Variable()  # Amount invested in the government bond

# Define the objective function
yield_total = 0.04 * x_corp + 0.03 * x_gov
objective = cp.Maximize(yield_total)

# Define the constraints
constraints = [
    x_corp + x_gov <= 100000,             # Total investment constraint
    (3 * x_corp + 4 * x_gov)  <= 360000,  # Average maturity constraint
    (2 * x_corp + x_gov)  <= 150000,      # Average rating constraint
    x_corp >= 0,                          # Non-negativity constraint for corporate bond
    x_gov >= 0                            # Non-negativity constraint for government bond
]

# Solve the problem
problem = cp.Problem(objective, constraints)
problem.solve()

# Display the results
print(f"Invest in Corporate Bond: ${x_corp.value:.2f}")
print(f"Invest in Government Bond: ${x_gov.value:.2f}")



Invest in Corporate Bond: $50000.00
Invest in Government Bond: $50000.00


# PYOMO

Pyomo is one of the popular open-source libraries that can be used to solve a wide array of optimization problems. It is developed by Sandia National Laboratories and can interface with many solvers. A key disadvantage, however, is its complexity which leads to a steeper learning curve of the language, compared to others.

In [22]:
!pip install pyomo

  Obtaining dependency information for pyomo from https://files.pythonhosted.org/packages/a3/70/ec7ff52c19632395263599e91ad0e8c6fdc9f793f254941e701674dc7326/Pyomo-6.6.1-cp38-cp38-macosx_10_15_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.3/5.3 MB 3.1 MB/s eta 0:00:0000:0100:01m
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 23.3 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [28]:
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory, NonNegativeReals, maximize

# Create a model
model = ConcreteModel()

# Define the decision variables
model.x_corp = Var(within=NonNegativeReals)  # Corporate bond investment
model.x_gov = Var(within=NonNegativeReals)   # Government bond investment

# Define the objective function
def objective_rule(m):
    return 0.04 * m.x_corp + 0.03 * m.x_gov

model.profit = Objective(rule=objective_rule, sense=maximize)

# Define constraints
def total_investment_rule(m):
    return m.x_corp + m.x_gov <= 100000

model.total_investment = Constraint(rule=total_investment_rule)

def avg_maturity_rule(m):
    return (3 * m.x_corp + 4 * m.x_gov) <= 3.6 * (m.x_corp + m.x_gov)  # Reformulated to avoid division

model.avg_maturity = Constraint(rule=avg_maturity_rule)

def avg_rating_rule(m):
    return (2 * m.x_corp + m.x_gov) <= 1.5 * (m.x_corp + m.x_gov)      # Reformulated to avoid division

model.avg_rating = Constraint(rule=avg_rating_rule)

# Solve
solver = SolverFactory('glpk')
solver.solve(model)

# Display results
print(f"Invest in Corporate Bond: ${model.x_corp()}")
print(f"Invest in Government Bond: ${model.x_gov()}")


Invest in Corporate Bond: $50000.0
Invest in Government Bond: $50000.0
